In [22]:
import cv2
import torch
import numpy as np
from ultralytics import YOLO
import torchvision.transforms as transforms

In [14]:
CONFIDENCE_THRESHOLD = 0.5  # Confidence threshold for detecting persons
IOU_THRESHOLD = 0.5  # IoU threshold for tracking

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [16]:
model = YOLO('yolov8m.pt')
model.to(device)

100%|██████████| 49.7M/49.7M [00:10<00:00, 4.92MB/s]


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_

In [17]:
# Helper function to calculate Intersection over Union (IoU)
def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1_2, y1_2, x2_2, y2_2 = box2

    # Compute the coordinates of the intersection rectangle
    xi1 = max(x1, x1_2)
    yi1 = max(y1, y1_2)
    xi2 = min(x2, x2_2)
    yi2 = min(y2, y2_2)

    # Compute the area of intersection
    intersection_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)

    # Compute the area of both bounding boxes
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_2 - x1_2) * (y2_2 - y1_2)

    # Compute the union area
    union_area = box1_area + box2_area - intersection_area

    # Compute the IoU
    iou = intersection_area / union_area
    return iou


In [18]:
# Function to assign unique IDs
class PersonTracker:
    def __init__(self):
        self.persons = {}  # Stores ID -> (bbox)
        self.next_id = 0   # Counter for the next unique ID

    def track(self, bboxes):
        new_persons = []
        
        for bbox in bboxes:
            assigned = False
            for person_id, person_bbox in self.persons.items():
                # Compare IoU to check if it's the same person
                iou = calculate_iou(person_bbox, bbox)
                if iou >= IOU_THRESHOLD:
                    # Update the person with the new bounding box
                    self.persons[person_id] = bbox
                    new_persons.append(person_id)
                    assigned = True
                    break

            if not assigned:
                # Assign a new unique ID
                self.persons[self.next_id] = bbox
                new_persons.append(self.next_id)
                self.next_id += 1
        
        return new_persons

# Initialize tracker
tracker = PersonTracker()

In [20]:
video_path = r'C:\Users\ahmad\Downloads\853889-hd_1920_1080_25fps.mp4'  # Path to your MP4 file
cap = cv2.VideoCapture(video_path)

In [25]:
import torch
import cv2
import torch.nn.functional as F

CONFIDENCE_THRESHOLD = 0.5  # Set this to the confidence level you desire

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to a tensor, normalize, and move it to the same device as the model
    frame_tensor = torch.from_numpy(frame).float().to(device)
    frame_tensor = frame_tensor.permute(2, 0, 1) / 255.0  # Normalize and change shape to (C, H, W)

    # Resize the input frame to 640x640 using torch.nn.functional.interpolate
    frame_tensor_resized = F.interpolate(frame_tensor.unsqueeze(0), size=(640, 640), mode='bilinear', align_corners=False)

    # Run YOLOv8 inference on the resized frame
    results = model(frame_tensor_resized)

    # Extract bounding boxes and class labels
    bboxes = []
    confidences = []
    classes = results.xywh[0][:, -1].tolist()  # Class labels
    scores = results.xywh[0][:, 4].tolist()   # Confidence scores
    for i in range(len(classes)):
        if classes[i] == 0 and scores[i] >= CONFIDENCE_THRESHOLD:  # Class 0 is 'person' in COCO
            x1, y1, x2, y2 = map(int, results.xywh[0][i][:4])  # Bounding box coordinates
            bboxes.append((x1, y1, x2, y2))

    # Track persons and assign IDs
    person_ids = tracker.track(bboxes)

    # Draw bounding boxes and IDs
    for i, bbox in enumerate(bboxes):
        x1, y1, x2, y2 = bbox
        person_id = person_ids[i]

        # Draw bounding box and ID
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {person_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with bounding boxes and IDs
    cv2.imshow("TrackNet-X", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


NotImplementedError: Could not run 'torchvision::nms' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'torchvision::nms' is only available for these backends: [CPU, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

CPU: registered at C:\actions-runner\_work\vision\vision\pyt

In [4]:
import cv2
import torch

# Initialize YOLO model and set to detect only 'person'
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
model.classes = [0]  # Class 0 is 'person' in COCO

# Video capture from .mp4 file
cap = cv2.VideoCapture(r'C:\Users\ahmad\Downloads\853889-hd_1920_1080_25fps.mp4')
# Tracking configuration
iou_threshold = 0.3  # Minimum IoU to consider as the same person
frame_retention = 30  # Number of frames to retain ID if person is not detected

# Dictionary to store tracked persons and frame retention counter
tracked_persons = {}  # {ID: {'bbox': (x1, y1, x2, y2), 'frames': 0}}

# Unique ID counter
person_id_counter = 1

def calculate_iou(box1, box2):
    # Calculate Intersection over Union (IoU) between two boxes
    x1, y1, x2, y2 = box1
    x1_p, y1_p, x2_p, y2_p = box2
    xi1, yi1, xi2, yi2 = max(x1, x1_p), max(y1, y1_p), min(x2, x2_p), min(y2, y2_p)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_p - x1_p) * (y2_p - y1_p)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection
    results = model(frame)  # Frame is passed directly as BGR NumPy array
    detections = results.pandas().xyxy[0]  # Get detection data

    # Filter only 'person' detections and extract bounding boxes
    new_bboxes = []
    for _, row in detections.iterrows():
        if row['name'] == 'person':  # Ensure only 'person' detections
            x1, y1, x2, y2 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
            new_bboxes.append((x1, y1, x2, y2))

    # Dictionary to keep updated persons in this frame
    updated_tracked_persons = {}

    # Check each new detection against tracked persons
    for bbox in new_bboxes:
        best_iou = 0
        best_match_id = None

        # Find the best match from tracked persons
        for person_id, person_data in tracked_persons.items():
            iou = calculate_iou(bbox, person_data['bbox'])
            if iou > best_iou and iou > iou_threshold:
                best_iou = iou
                best_match_id = person_id

        if best_match_id is None:
            # No match found, assign new ID
            best_match_id = person_id_counter
            person_id_counter += 1

        # Update tracked persons with this ID and reset the frame retention counter
        updated_tracked_persons[best_match_id] = {'bbox': bbox, 'frames': 0}

    # Update the tracked persons dictionary
    for person_id, person_data in tracked_persons.items():
        if person_id not in updated_tracked_persons:
            person_data['frames'] += 1
            if person_data['frames'] < frame_retention:
                updated_tracked_persons[person_id] = person_data  # Keep the person if within retention limit

    tracked_persons = updated_tracked_persons

    # Draw bounding boxes and IDs
    for person_id, person_data in tracked_persons.items():
        x1, y1, x2, y2 = person_data['bbox']
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID: {person_id}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with bounding boxes and IDs
    cv2.imshow("Person Detection and Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Using cache found in C:\Users\ahmad/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-11-9 Python-3.10.0 torch-2.5.1+cu118 CUDA:0 (NVIDIA GeForce RTX 2060, 6144MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
C:\Users\ahmad/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


NotImplementedError: Could not run 'torchvision::nms' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'torchvision::nms' is only available for these backends: [CPU, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

CPU: registered at C:\actions-runner\_work\vision\vision\pyt